In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pymongo import MongoClient
import pandas as pd
import requests
import os
import folium
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = MongoClient("mongodb://localhost/datamad0320")
db = client.get_database()

In [3]:
!mongoimport --db=datamad0320 --collection=ejercicios --drop ./Output/companies.json


2020-06-02T11:56:35.794+0200	connected to: mongodb://localhost/
2020-06-02T11:56:35.794+0200	dropping: datamad0320.ejercicios
2020-06-02T11:56:38.794+0200	[########................] datamad0320.ejercicios	27.6MB/74.6MB (37.0%)
2020-06-02T11:56:41.794+0200	[################........] datamad0320.ejercicios	51.4MB/74.6MB (68.9%)
2020-06-02T11:56:44.435+0200	[########################] datamad0320.ejercicios	74.6MB/74.6MB (100.0%)
2020-06-02T11:56:44.435+0200	18801 document(s) imported successfully. 0 document(s) failed to import.


In [4]:
all_offices = list(db.ejercicios.find({},{"offices":1,"name":1,"founded_year":1,"category_code":1}))


In [5]:
companias = pd.DataFrame (all_offices )
companias = companias.explode("offices")
companias.head()


,_id,name,category_code,founded_year,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,"{'description': '', 'address1': '710 - 2nd Ave..."
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,2005.0,"{'description': '', 'address1': '270 Lafayette..."
1,52cdef7c4bab8bd675297d90,Postini,web,1999.0,"{'description': None, 'address1': '959 Skyway ..."
2,52cdef7c4bab8bd675297d91,Geni,web,2006.0,"{'description': 'Headquarters', 'address1': '9..."
3,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,2005.0,"{'description': '', 'address1': 'Suite 200', '..."
...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,"{'description': '', 'address1': '', 'address2'..."
18797,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN
18798,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,2000.0,"{'description': '', 'address1': '1900 Internat..."
18799,52cdef7f4bab8bd67529c6f7,AfterLogic,software,2002.0,"{'description': 'Livingston', 'address1': 'P.O..."


In [7]:
limpias_offices = companias.apply(buscaroficinaGeoPoint,axis=1, result_type="expand")
limpias_offices.columns = ["office","clean_state"]
limpias_offices

,office,clean_state
0,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
1,"{'type': 'Point', 'coordinates': [-122.247573,...",success
2,"{'type': 'Point', 'coordinates': [-118.393064,...",success
3,None,Invalid lat lat and long
...,...,...
18796,"{'type': 'Point', 'coordinates': [-86.816068, ...",success
18797,None,No office
18798,"{'type': 'Point', 'coordinates': [-86.7613019,...",success
18799,"{'type': 'Point', 'coordinates': [-74.3235539,...",success


In [8]:
companias_limpias= pd.concat([companias,limpias_offices], axis=1)

In [9]:
companias_limpias = companias_limpias[["name","category_code","office","clean_state","founded_year"]]

In [10]:
companias_limpias.clean_state.value_counts()

success                     10834
Invalid lat lat and long     5871
No office                    5057
Name: clean_state, dtype: int64

In [11]:
companias_limpias = companias_limpias.drop(companias_limpias.index[companias_limpias["clean_state"]=="No office",])

In [107]:
companias_limpias.clean_state.value_counts()
companias_limpias.head()

,name,category_code,Lugares,clean_state,founded_year
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success,2005.0
1,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,2005.0
2,Postini,web,"{'type': 'Point', 'coordinates': [-122.247573,...",success,1999.0
3,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,2006.0
4,Omnidrive,network_hosting,None,Invalid lat lat and long,2005.0


In [13]:
companias_limpias.rename(columns={'office':'Lugares'}, inplace=True)

In [14]:
companias_limpias = companias_limpias.reset_index()

In [15]:
companias_limpias = companias_limpias.drop(['index'], axis=1)

In [108]:
companias_limpias.head()

,name,category_code,Lugares,clean_state,founded_year
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success,2005.0
1,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,2005.0
2,Postini,web,"{'type': 'Point', 'coordinates': [-122.247573,...",success,1999.0
3,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,2006.0
4,Omnidrive,network_hosting,None,Invalid lat lat and long,2005.0


In [109]:
companias_limpias = companias_limpias.dropna(subset=["founded_year"])
companias_limpias.head()

,name,category_code,Lugares,clean_state,founded_year
0,Wetpaint,web,"{'type': 'Point', 'coordinates': [-122.333253,...",success,2005.0
1,Wetpaint,web,"{'type': 'Point', 'coordinates': [-73.9964312,...",success,2005.0
2,Postini,web,"{'type': 'Point', 'coordinates': [-122.247573,...",success,1999.0
3,Geni,web,"{'type': 'Point', 'coordinates': [-118.393064,...",success,2006.0
4,Omnidrive,network_hosting,None,Invalid lat lat and long,2005.0


In [19]:
companias_limpias.to_json("Input/Companias.json",orient="records")

In [20]:
!mongoimport --db=datamad0320 --collection=CompaniasLimpias --drop --jsonArray ./Input/Companias.json

2020-06-02T11:56:56.762+0200	connected to: mongodb://localhost/
2020-06-02T11:56:56.763+0200	dropping: datamad0320.CompaniasLimpias
2020-06-02T11:56:57.828+0200	14057 document(s) imported successfully. 0 document(s) failed to import.


In [25]:
#, maxDist=1000
def getOfficeNear(address, maxDist=3000):
    point = geocodeMadrid(address)
    return {
       "office": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }


In [26]:
def getBuscar(address, maxDist=1000):
    point = geocodeMadrid(address)
    return {
       "Lugares": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }


In [27]:
query = getOfficeNear("RETIRO Madrid")
query

<Response [200]>


{'office': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-3.67744, 40.40816]},
   '$maxDistance': 3000}}}

In [50]:
#cur=(list(db.CompaniasLimpias.find({"Lugares": {"$near":{"$geometry":{"type":"Point","coordinates":[ 40.40816,-3.67744]},"$maxDistance":1000}}})))

#cur = db.NewYorkCompanis.find(query.update({"clean_state":"success"}), {"_id":0})
cur =( db.CompaniasLimpias.find({"Lugares":{'$near': {'$geometry': {'type': 'Point', 'coordinates':[-3.67744, 40.40816]},'$maxDistance': 5000}}}) )
                        #{"_id":0}})

result = cur
list(result)

[{'_id': ObjectId('5ed6226943f93d00091fa347'),
  'name': 'Digital Assets Deployment',
  'category_code': 'ecommerce',
  'Lugares': {'type': 'Point', 'coordinates': [-3.678121, 40.42046]},
  'clean_state': 'success',
  'founded_year': 2006.0},
 {'_id': ObjectId('5ed6226943f93d00091fb160'),
  'name': 'Busuu',
  'category_code': 'social',
  'Lugares': {'type': 'Point', 'coordinates': [-3.6830316, 40.4225487]},
  'clean_state': 'success',
  'founded_year': 2008.0},
 {'_id': ObjectId('5ed6226943f93d00091fbbdb'),
  'name': 'Tuenti Technologies',
  'category_code': 'social',
  'Lugares': {'type': 'Point', 'coordinates': [-3.696277, 40.4158236]},
  'clean_state': 'success',
  'founded_year': 2006.0},
 {'_id': ObjectId('5ed6226943f93d00091fc2b3'),
  'name': 'Tagmore Solutions',
  'category_code': 'mobile',
  'Lugares': {'type': 'Point', 'coordinates': [-3.6883074, 40.4226918]},
  'clean_state': 'success',
  'founded_year': 2007.0},
 {'_id': ObjectId('5ed6226943f93d00091fbe0b'),
  'name': 'ASPge

In [20]:
def buscarOficinasCerca(address, maxDist=1000):
    point = geocode(address)
    return {
       "office": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }

In [85]:
guarderiasretiro = ExtractDireccion("madrid retiro","guarderias", 2)
aeropuerto = ExtractDireccion("madrid","aeropuerto", 1)
starbucks = ExtractDireccion("madrid retiro","starbucks", 1)

In [110]:
guarderiasretiroDatos = pd.DataFrame(guarderiasretiro)
aeropuertoDatos = pd.DataFrame(aeropuerto)
starbucksDatos = pd.DataFrame(starbucks)

In [111]:
#Renombro las columnas con name y adress
aeropuertoDatos = aeropuertoDatos.rename(columns={0:'name',1:'adress'})
starbucksDatos= starbucksDatos.rename(columns={0:'name',1:'adress'})
guarderiasretiroDatos = guarderiasretiroDatos.rename(columns={0:'name',1:'adress'})

In [114]:
display(guarderiasretiroDatos.head(2),starbucksDatos.head(2),aeropuertoDatos.head(2))

,name,adress
0,Jaizkibel,"Calle de José Del Hierro, 29"
1,Escuela Infantil Pipo’s,"Calle de Cáceres, 50"


,name,adress
0,Starbucks,"Calle de José Ortega y Gasset, 29"
1,Starbucks,"Calle Doctor Drumen, 4"


,name,adress
0,Aeropuerto de Madrid-Barajas Adolfo Suárez,"Avenida de la Hispanidad, s/n"
1,Taxi Aeropuerto Madrid,"Gran Vía, 30"


In [71]:
aeropuertoDatos["coordenadas"]= geocodeGoogleMasivo(aeropuertoDatos)

In [115]:
guarderiasretiroDatos["coordenadas"] = geocodeGoogleMasivo(guarderiasretiroDatos)
guarderiasretiroDatos

,name,adress,coordenadas
0,Jaizkibel,"Calle de José Del Hierro, 29","{'type': 'Point', 'coordinates': [-3.6455125, ..."
1,Escuela Infantil Pipo’s,"Calle de Cáceres, 50","{'type': 'Point', 'coordinates': [-3.7000098, ..."
2,El Bosque Encantado,"Calle de Antonio López, 246","{'type': 'Point', 'coordinates': [-3.6959297, ..."
3,Guarderia Arco de Colores,Calle Gumersindo Azcarate 32,"{'type': 'Point', 'coordinates': [-3.7097628, ..."
4,Guarderia Infantil Primeros Pasos,"Calle Campo de la Paloma, 12","{'type': 'Point', 'coordinates': [-3.6465462, ..."
5,Estrella Amdal,"Calle Francisco Vitoria, 4","{'type': 'Point', 'coordinates': [-3.674009, 4..."
6,Rocho Carmona Maria del Carmen,"Calle Reyes Magos, 6","{'type': 'Point', 'coordinates': [-3.674691499..."
7,Cei Fantasía,Calle Justa García 12,"{'type': 'Point', 'coordinates': [-3.7095009, ..."
8,Niro,"Calle Andres Torrejon, 18","{'type': 'Point', 'coordinates': [-3.6818551, ..."
9,Teddy Centro Infantil,"Calle Leon GIL de Palacio, 5 - LC 10","{'type': 'Point', 'coordinates': [-3.6790747, ..."


In [116]:
starbucksDatos["coordenadas"] =  geocodeGoogleMasivo(starbucksDatos)
starbucksDatos

,name,adress,coordenadas
0,Starbucks,"Calle de José Ortega y Gasset, 29","{'type': 'Point', 'coordinates': [-3.6830042, ..."
1,Starbucks,"Calle Doctor Drumen, 4","{'type': 'Point', 'coordinates': [-3.6931363, ..."
2,Starbucks,"Calle de la Montera, 8","{'type': 'Point', 'coordinates': [-3.7027745, ..."
3,Starbucks,"Calle del Conde de Peñalver, 10","{'type': 'Point', 'coordinates': [-3.6753894, ..."
4,Starbucks,"Calle Arenal, 14","{'type': 'Point', 'coordinates': [-3.7061836, ..."
5,Starbucks,"Plaza Cánovas del Castillo, 5","{'type': 'Point', 'coordinates': [-3.6950046, ..."
6,Starbucks,"Calle de Ciudad Rodrigo, 5","{'type': 'Point', 'coordinates': [-3.7084449, ..."
7,Starbucks,"Calle Gran Vía, 30","{'type': 'Point', 'coordinates': [-3.702459999..."
8,Starbucks - CERRADO,"Plaza de Callao, 4","{'type': 'Point', 'coordinates': [-3.7059615, ..."
9,Starbucks,"Calle Velázquez, 32","{'type': 'Point', 'coordinates': [-3.6839483, ..."


In [117]:
list(starbucksDatos.coordenadas)

[{'type': 'Point', 'coordinates': [-3.6830042, 40.43061549999999]},
 {'type': 'Point', 'coordinates': [-3.6931363, 40.4091219]},
 {'type': 'Point', 'coordinates': [-3.7027745, 40.4175555]},
 {'type': 'Point', 'coordinates': [-3.6753894, 40.426058]},
 {'type': 'Point', 'coordinates': [-3.7061836, 40.4173115]},
 {'type': 'Point', 'coordinates': [-3.6950046, 40.4152043]},
 {'type': 'Point', 'coordinates': [-3.7084449, 40.4156892]},
 {'type': 'Point', 'coordinates': [-3.702459999999999, 40.4202362]},
 {'type': 'Point', 'coordinates': [-3.7059615, 40.42055980000001]},
 {'type': 'Point', 'coordinates': [-3.6839483, 40.4249447]}]

In [73]:
#Elimino las filas que no coinciden con un aeropuerto
aeropuertoDatos.drop(aeropuertoDatos.index[[0,1,4,5,6,7]],inplace=True)

In [121]:
starbucksDatos["Coordenadas"] = geocodeStarbucks

In [ ]:
starbucksDatos.drop(aeropuertoDatos.index[[4,5]],inplace=True)

In [130]:
starbucksDatos

,name,adress
0,Starbucks,"Calle de José Ortega y Gasset, 29"
1,Starbucks,"Calle Doctor Drumen, 4"
2,Starbucks,"Calle de la Montera, 8"
3,Starbucks,"Calle del Conde de Peñalver, 10"
4,Starbucks,"Calle Arenal, 14"
5,Starbucks,"Plaza Cánovas del Castillo, 5"
6,Starbucks,"Calle de Ciudad Rodrigo, 5"
7,Starbucks,"Calle Gran Vía, 30"
8,Starbucks - CERRADO,"Plaza de Callao, 4"
9,Starbucks,"Calle Velázquez, 32"


In [106]:
display((aeropuertoDatos))

,name,adress,Lugares
0,Aeropuerto de Madrid-Barajas Adolfo Suárez,"Avenida de la Hispanidad, s/n","{'type': 'Point', 'coordinates': [-3.5828701, ..."
1,Taxi Aeropuerto Madrid,"Gran Vía, 30","{'type': 'Point', 'coordinates': [-3.702459999..."
2,Objetos Perdidos Aeropuerto de Madrid Barajas,"Calle Aeropuerto de Barajas, s/n","{'type': 'Point', 'coordinates': [-3.5675982, ..."
3,Iberia Puente Aereo,Aeropuerto de Barajas Terminal 4,"{'type': 'Point', 'coordinates': [-3.5917845, ..."
4,Aeropuerto de Madrid-Torrejon,"Calle Peonias, 2","{'type': 'Point', 'coordinates': [-3.6179092, ..."
5,Aeropuerto de Madrid - Cuatro Vientos,"Carretera Barrio de la Fortuna, s/n","{'type': 'Point', 'coordinates': [-3.7714895, ..."
6,Vip Sala Terminal 2,"Avenida General, 43","{'type': 'Point', 'coordinates': [-3.579875, 4..."
7,Aeropuerto De Madrid-Barajas Adolfo Suárez Ter...,Via Lusitana 27,"{'type': 'Point', 'coordinates': [-3.7315917, ..."


In [ ]:
starbucksDatos

In [ ]:
localizaciones

In [83]:
#Elimino la fila con dos direcciones
#guarderiasretiroDatos.drop(guarderiasretiroDatos.index[[17]],inplace=True)

In [202]:
#Elimino la fila con el Starbucks cerrado
#starbucksDatos.drop(starbucksDatos.index[[4]],inplace=True)
display(aeropuertoDatos,guarderiasretiroDatos)

,name,adress
0,John F Kennedy International Airport - JFK,Van Wyck & JFK Expressway
1,Newark Liberty International Airport - EWR,3 Brewster Rd
2,Teterboro Airport,510 Industrial Ave
3,LaGuardia Airport Terminal C,1 Central Terminal Dr
4,LaGuardia Airport - LGA,94-00 Grand Central Pkwy
5,TWA Flight Center,Jfk Airport
6,TSA Checkpoint Terminal C/2,43 Olympia Dr
7,TSA Checkpoint Terminal C/1,3100 47th Ave


,name,adress
0,Preschool of America,39 Eldridge St
1,Trinity Preschool,28 Washington St
2,The Learning Experience - Battery Park,80 Beekman St
3,Bright Beginnings NYC,228 Bushwick Ave
4,Sunset Park Playhouse,1677 Lexington Ave
5,Creative Academy Daycare Center,67 E 11th St
6,Friends and Family Day Care,451 7th Ave
7,Kidville,4616 4th Ave
8,Chickpeas Child Care Center,1485 Bloomfield St
9,Sunset Park Children’s School,406 W 55th St


In [53]:
import time
def masivoGeocodeAeropuerto(dataframe):
    puntos_geocode=[]
    for direccion in dataframe.name:
        time.sleep(1)
        puntos_geocode.append(geocode(direccion))
    return puntos_geocode


In [29]:

def geocode(address):
    '''resp = requests.get("https://geocode.xyz/51.4647,0.0079?json=1&auth=your auth code")
    Use geocode api to do forward geocoding. https://geocode.xyz/api
    '''
    res = requests.get(f"https://geocode.xyz/{address},USA, New York?region=US?json=1&auth={api_key_Geo}",params={"json":1})
    #res = requests.get(f"https://geocode.xyz/{address}",params={"json":1})
    data = res.json()
    print(res)
    # Return as GeoJSON -> https://geojson.org/
    return {
        "type":"Point",
        "coordinates": [float(data["latt"]), float(data["longt"])]
    }

In [30]:
restvega = masivoGeocode(restvegaDatos)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [31]:
print(restvega )

[{'type': 'Point', 'coordinates': [40.76513, -73.9659]}, {'type': 'Point', 'coordinates': [40.58277, -74.10319]}, {'type': 'Point', 'coordinates': [40.72052, -73.99658]}, {'type': 'Point', 'coordinates': [40.76201, -73.9874]}, {'type': 'Point', 'coordinates': [40.68908, -73.9586]}, {'type': 'Point', 'coordinates': [40.71419, -73.9887]}, {'type': 'Point', 'coordinates': [40.72841, -73.98358]}, {'type': 'Point', 'coordinates': [40.58774, -74.08615]}, {'type': 'Point', 'coordinates': [40.7473, -73.91212]}, {'type': 'Point', 'coordinates': [40.68646, -73.986]}, {'type': 'Point', 'coordinates': [40.67241, -73.95734]}, {'type': 'Point', 'coordinates': [40.72263, -73.98486]}, {'type': 'Point', 'coordinates': [40.81238, -73.84744]}, {'type': 'Point', 'coordinates': [40.7301, -74.00357]}, {'type': 'Point', 'coordinates': [40.71417, -73.98748]}, {'type': 'Point', 'coordinates': [40.61521, -74.09787]}, {'type': 'Point', 'coordinates': [40.65794, -73.95049]}, {'type': 'Point', 'coordinates': [40.6

In [34]:
#Con la función Geocode pido los puntos geodésicos
#aeropuertos= masivoGeocodeAeropuerto(aeropuertoDatos)

#starbucksgeo = masivoGeocode(starbucksDatos)

[{'type': 'Point', 'coordinates': [40.74313, -73.7567]}, {'type': 'Point', 'coordinates': [40.70567, -74.01538]}, {'type': 'Point', 'coordinates': [40.85365, -74.06037]}, {'type': 'Point', 'coordinates': [40.72693, -73.99992]}, {'type': 'Point', 'coordinates': [40.72693, -73.99992]}, {'type': 'Point', 'coordinates': [40.57153, -74.14344]}, {'type': 'Point', 'coordinates': [40.76031, -112.00659]}, {'type': 'Point', 'coordinates': [40.76031, -112.00659]}]


In [ ]:
print( aeropuertos)

In [36]:
guarderias = masivoGeocode(guarderiasretiroDatos)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [62]:
#Esto no lo metas es cargar los archivos
with open('./Input/guarderiasNY.json', 'r') as myfile:
       guarderiasretiroDatos=myfile.read()

with open('./Input/RestVeganoNY.json', 'r') as tofile:
       starbucksDatos= tofile.read()
with open('./Input/aeropuertosfinal.json', 'r') as sufile:
       aeropuertoDatos = sufile.read()



In [37]:
print(guarderias)

[{'type': 'Point', 'coordinates': [40.71569, -73.99334]}, {'type': 'Point', 'coordinates': [40.73141, -73.99697]}, {'type': 'Point', 'coordinates': [40.7081, -74.0034]}, {'type': 'Point', 'coordinates': [40.70792, -73.94004]}, {'type': 'Point', 'coordinates': [40.79171, -73.9465]}, {'type': 'Point', 'coordinates': [40.73292, -73.99241]}, {'type': 'Point', 'coordinates': [40.7506, -73.99091]}, {'type': 'Point', 'coordinates': [40.7205, -73.97556]}, {'type': 'Point', 'coordinates': [40.60918, -74.18833]}, {'type': 'Point', 'coordinates': [40.76671, -73.98689]}, {'type': 'Point', 'coordinates': [40.77646, -73.94435]}, {'type': 'Point', 'coordinates': [40.77732, -73.95215]}, {'type': 'Point', 'coordinates': [40.66996, -73.93161]}, {'type': 'Point', 'coordinates': [40.59699, -74.06358]}, {'type': 'Point', 'coordinates': [40.74251, -73.90733]}, {'type': 'Point', 'coordinates': [40.71274, -73.99455]}, {'type': 'Point', 'coordinates': [40.78776, -73.95334]}, {'type': 'Point', 'coordinates': [4

In [66]:

df_restvegageo = pd.DataFrame(restvega)
df_guarderiasgeo= pd.DataFrame(guarderias)
df_aeropuertosgeo= pd.DataFrame(aeropuertos)


In [67]:
def easyLatLng2(row):
    of = row
    return {
        "latitude":of["coordinates"][1],
        "longitude":of["coordinates"][0]
    }


In [68]:
#Organizo el indice de guarderiasDatos y uno los DataFrame.
guarderiasretiroDatos = guarderiasretiroDatos.reset_index()
df_guarderias = pd.concat([df_guarderiasgeo, df_guarderiasgeo.apply(easyLatLng2, axis=1, result_type="expand")], axis=1)
df_guarderias2 = pd.concat([guarderiasretiroDatos,df_guarderias], axis=1) 


In [69]:
#Organizo el indice de aeropuertoDatos y uno los DataFrame.
aeropuertoDatos = aeropuertoDatos.reset_index()
df_aeropuertos = pd.concat([df_aeropuertosgeo, df_aeropuertosgeo.apply(easyLatLng2, axis=1, result_type="expand")], axis=1)
df_aeropuertos2 = pd.concat([aeropuertoDatos,df_aeropuertos], axis=1) 


,level_0,index,name,adress,type,coordinates,latitude,longitude,latitude,longitude
0,0,0,Loco Coco,835 Lexington Ave,Point,"[40.76513, -73.9659]",-73.96590,40.76513,-73.96590,40.76513
1,1,1,Hungry Angelina,117 Adams St,Point,"[40.58277, -74.10319]",-74.10319,40.58277,-74.10319,40.58277
2,2,2,October,384 Broome St,Point,"[40.72052, -73.99658]",-73.99658,40.72052,-73.99658,40.72052
3,3,3,MTHR Vegan,329 W 49th St,Point,"[40.76201, -73.9874]",-73.98740,40.76201,-73.98740,40.76201
4,4,4,Vital Vegan Cafe,1217 Teaneck Rd,Point,"[40.68908, -73.9586]",-73.95860,40.68908,-73.95860,40.68908
5,5,5,Petisco Vegano,189 E Broadway,Point,"[40.71419, -73.9887]",-73.98870,40.71419,-73.98870,40.71419
6,6,6,Vegan Love,263 E 10th St,Point,"[40.72841, -73.98358]",-73.98358,40.72841,-73.98358,40.72841
7,7,7,Vegans of Seitan,669 Raritan Rd,Point,"[40.58774, -74.08615]",-74.08615,40.58774,-74.08615,40.58774
8,8,8,Zona Rosa Taqueria,6021 39th Ave,Point,"[40.7473, -73.91212]",-73.91212,40.74730,-73.91212,40.74730
9,9,9,Plant Base,370 Pacific Ave,Point,"[40.68646, -73.986]",-73.98600,40.68646,-73.98600,40.68646


In [70]:
#Organizo el indice de starbucksDatos y uno los DataFrame.
restvegaDatos = restvegaDatos.reset_index()
df_restvegageo = pd.concat([df_restvegageo, df_restvegageo.apply(easyLatLng2, axis=1, result_type="expand")], axis=1)
df_restvegageo2 = pd.concat([restvegaDatos,df_restvegageo], axis=1) 

In [71]:
#cambio a lugares el nombre de la columna
df_restvegageo3 =pd.concat([df_restvegageo2,(pd.DataFrame({'Lugares': restvega},
                                                      index= df_restvegageo2.index ))],axis=1)

In [72]:
df_guarderias3 =pd.concat([df_guarderias2,(pd.DataFrame({'Lugares': guarderias},
                                                        index= df_guarderias2.index ))],axis=1)

In [73]:
df_aeropuertos3 =pd.concat([df_aeropuertos2,(pd.DataFrame({'Lugares': aeropuertos},
                                                        index= df_aeropuertos2.index ))],axis=1)
df_aeropuertos3


,index,name,adress,type,coordinates,latitude,longitude,Lugares
0,0,John F Kennedy International Airport - JFK,Van Wyck & JFK Expressway,Point,"[40.74313, -73.7567]",-73.75670,40.74313,"{'type': 'Point', 'coordinates': [40.74313, -7..."
1,1,Newark Liberty International Airport - EWR,3 Brewster Rd,Point,"[40.70567, -74.01538]",-74.01538,40.70567,"{'type': 'Point', 'coordinates': [40.70567, -7..."
2,2,Teterboro Airport,510 Industrial Ave,Point,"[40.85365, -74.06037]",-74.06037,40.85365,"{'type': 'Point', 'coordinates': [40.85365, -7..."
3,3,LaGuardia Airport Terminal C,1 Central Terminal Dr,Point,"[40.72693, -73.99992]",-73.99992,40.72693,"{'type': 'Point', 'coordinates': [40.72693, -7..."
4,4,LaGuardia Airport - LGA,94-00 Grand Central Pkwy,Point,"[40.72693, -73.99992]",-73.99992,40.72693,"{'type': 'Point', 'coordinates': [40.72693, -7..."
5,5,TWA Flight Center,Jfk Airport,Point,"[40.57153, -74.14344]",-74.14344,40.57153,"{'type': 'Point', 'coordinates': [40.57153, -7..."
6,6,TSA Checkpoint Terminal C/2,43 Olympia Dr,Point,"[40.76031, -112.00659]",-112.00659,40.76031,"{'type': 'Point', 'coordinates': [40.76031, -1..."
7,7,TSA Checkpoint Terminal C/1,3100 47th Ave,Point,"[40.76031, -112.00659]",-112.00659,40.76031,"{'type': 'Point', 'coordinates': [40.76031, -1..."


In [75]:
df_restvegageo3

,index,name,adress,type,coordinates,latitude,longitude,Lugares
0,0,Loco Coco,835 Lexington Ave,Point,"[40.76513, -73.9659]",-73.96590,40.76513,"{'type': 'Point', 'coordinates': [40.76513, -7..."
1,1,Hungry Angelina,117 Adams St,Point,"[40.58277, -74.10319]",-74.10319,40.58277,"{'type': 'Point', 'coordinates': [40.58277, -7..."
2,2,October,384 Broome St,Point,"[40.72052, -73.99658]",-73.99658,40.72052,"{'type': 'Point', 'coordinates': [40.72052, -7..."
3,3,MTHR Vegan,329 W 49th St,Point,"[40.76201, -73.9874]",-73.98740,40.76201,"{'type': 'Point', 'coordinates': [40.76201, -7..."
4,4,Vital Vegan Cafe,1217 Teaneck Rd,Point,"[40.68908, -73.9586]",-73.95860,40.68908,"{'type': 'Point', 'coordinates': [40.68908, -7..."
5,5,Petisco Vegano,189 E Broadway,Point,"[40.71419, -73.9887]",-73.98870,40.71419,"{'type': 'Point', 'coordinates': [40.71419, -7..."
6,6,Vegan Love,263 E 10th St,Point,"[40.72841, -73.98358]",-73.98358,40.72841,"{'type': 'Point', 'coordinates': [40.72841, -7..."
7,7,Vegans of Seitan,669 Raritan Rd,Point,"[40.58774, -74.08615]",-74.08615,40.58774,"{'type': 'Point', 'coordinates': [40.58774, -7..."
8,8,Zona Rosa Taqueria,6021 39th Ave,Point,"[40.7473, -73.91212]",-73.91212,40.74730,"{'type': 'Point', 'coordinates': [40.7473, -73..."
9,9,Plant Base,370 Pacific Ave,Point,"[40.68646, -73.986]",-73.98600,40.68646,"{'type': 'Point', 'coordinates': [40.68646, -7..."


In [76]:
#Elimino la columna de index que salio de más.
df_restvegageo3 = df_restvegageo3.drop(['index'], axis=1)
df_aeropuertos3 = df_aeropuertos3.drop(['index'], axis=1)
df_guarderias3 = df_guarderias3.drop(['index'], axis=1)
display(df_aeropuertos3,df_guarderias3 ,df_restvegageo3)

,name,adress,type,coordinates,latitude,longitude,Lugares
0,John F Kennedy International Airport - JFK,Van Wyck & JFK Expressway,Point,"[40.74313, -73.7567]",-73.75670,40.74313,"{'type': 'Point', 'coordinates': [40.74313, -7..."
1,Newark Liberty International Airport - EWR,3 Brewster Rd,Point,"[40.70567, -74.01538]",-74.01538,40.70567,"{'type': 'Point', 'coordinates': [40.70567, -7..."
2,Teterboro Airport,510 Industrial Ave,Point,"[40.85365, -74.06037]",-74.06037,40.85365,"{'type': 'Point', 'coordinates': [40.85365, -7..."
3,LaGuardia Airport Terminal C,1 Central Terminal Dr,Point,"[40.72693, -73.99992]",-73.99992,40.72693,"{'type': 'Point', 'coordinates': [40.72693, -7..."
4,LaGuardia Airport - LGA,94-00 Grand Central Pkwy,Point,"[40.72693, -73.99992]",-73.99992,40.72693,"{'type': 'Point', 'coordinates': [40.72693, -7..."
5,TWA Flight Center,Jfk Airport,Point,"[40.57153, -74.14344]",-74.14344,40.57153,"{'type': 'Point', 'coordinates': [40.57153, -7..."
6,TSA Checkpoint Terminal C/2,43 Olympia Dr,Point,"[40.76031, -112.00659]",-112.00659,40.76031,"{'type': 'Point', 'coordinates': [40.76031, -1..."
7,TSA Checkpoint Terminal C/1,3100 47th Ave,Point,"[40.76031, -112.00659]",-112.00659,40.76031,"{'type': 'Point', 'coordinates': [40.76031, -1..."


,name,adress,type,coordinates,latitude,longitude,Lugares
0,Preschool of America,39 Eldridge St,Point,"[40.71569, -73.99334]",-73.99334,40.71569,"{'type': 'Point', 'coordinates': [40.71569, -7..."
1,Trinity Preschool,28 Washington St,Point,"[40.73141, -73.99697]",-73.99697,40.73141,"{'type': 'Point', 'coordinates': [40.73141, -7..."
2,The Learning Experience - Battery Park,80 Beekman St,Point,"[40.7081, -74.0034]",-74.00340,40.70810,"{'type': 'Point', 'coordinates': [40.7081, -74..."
3,Bright Beginnings NYC,228 Bushwick Ave,Point,"[40.70792, -73.94004]",-73.94004,40.70792,"{'type': 'Point', 'coordinates': [40.70792, -7..."
4,Sunset Park Playhouse,1677 Lexington Ave,Point,"[40.79171, -73.9465]",-73.94650,40.79171,"{'type': 'Point', 'coordinates': [40.79171, -7..."
5,Creative Academy Daycare Center,67 E 11th St,Point,"[40.73292, -73.99241]",-73.99241,40.73292,"{'type': 'Point', 'coordinates': [40.73292, -7..."
6,Friends and Family Day Care,451 7th Ave,Point,"[40.7506, -73.99091]",-73.99091,40.75060,"{'type': 'Point', 'coordinates': [40.7506, -73..."
7,Kidville,4616 4th Ave,Point,"[40.7205, -73.97556]",-73.97556,40.72050,"{'type': 'Point', 'coordinates': [40.7205, -73..."
8,Chickpeas Child Care Center,1485 Bloomfield St,Point,"[40.60918, -74.18833]",-74.18833,40.60918,"{'type': 'Point', 'coordinates': [40.60918, -7..."
9,Sunset Park Children’s School,406 W 55th St,Point,"[40.76671, -73.98689]",-73.98689,40.76671,"{'type': 'Point', 'coordinates': [40.76671, -7..."


,name,adress,type,coordinates,latitude,longitude,Lugares
0,Loco Coco,835 Lexington Ave,Point,"[40.76513, -73.9659]",-73.96590,40.76513,"{'type': 'Point', 'coordinates': [40.76513, -7..."
1,Hungry Angelina,117 Adams St,Point,"[40.58277, -74.10319]",-74.10319,40.58277,"{'type': 'Point', 'coordinates': [40.58277, -7..."
2,October,384 Broome St,Point,"[40.72052, -73.99658]",-73.99658,40.72052,"{'type': 'Point', 'coordinates': [40.72052, -7..."
3,MTHR Vegan,329 W 49th St,Point,"[40.76201, -73.9874]",-73.98740,40.76201,"{'type': 'Point', 'coordinates': [40.76201, -7..."
4,Vital Vegan Cafe,1217 Teaneck Rd,Point,"[40.68908, -73.9586]",-73.95860,40.68908,"{'type': 'Point', 'coordinates': [40.68908, -7..."
5,Petisco Vegano,189 E Broadway,Point,"[40.71419, -73.9887]",-73.98870,40.71419,"{'type': 'Point', 'coordinates': [40.71419, -7..."
6,Vegan Love,263 E 10th St,Point,"[40.72841, -73.98358]",-73.98358,40.72841,"{'type': 'Point', 'coordinates': [40.72841, -7..."
7,Vegans of Seitan,669 Raritan Rd,Point,"[40.58774, -74.08615]",-74.08615,40.58774,"{'type': 'Point', 'coordinates': [40.58774, -7..."
8,Zona Rosa Taqueria,6021 39th Ave,Point,"[40.7473, -73.91212]",-73.91212,40.74730,"{'type': 'Point', 'coordinates': [40.7473, -73..."
9,Plant Base,370 Pacific Ave,Point,"[40.68646, -73.986]",-73.98600,40.68646,"{'type': 'Point', 'coordinates': [40.68646, -7..."


In [78]:
df_restvegageo3.to_json("./Input/restvegageoNY.json",orient="records")

In [79]:
df_aeropuertos3.to_json("./Input/aeropuertosNY.json",orient="records")

In [80]:
df_guarderias3.to_json("./Input/guarderiasNY.json",orient="records")

In [ ]:
def etiquetar(latitud, longitud,nombre):
    
    for latitud,longitud in zip(conGeo2.latitude,conGeo2.longitude):
    
        m = folium.Map(location=[latitud,longitud], zoom_start=12, tiles='New York')
    
    for names in no:
        folium.Marker(
            location=[latitud, longitud],
            popup= names,
            icon=folium.Icon(icon='info-sign')
             ).add_to(m)

    return m   
#for latitud,longitud in zip(conGeo2.latitude,conGeo2.longitude):
 #   etiquetar(latitud,longitud)
  #  m = folium.Map(location=[latitud,longitud], zoom_start=12, tiles='Stamen Terrain')
    

    
    
    
    # folium.Marker(
       #location=[latitud, longitud],
       # popup=names,
       #icon=folium.Icon(color='red', icon='info-sign')
       # ).add_to(m)

#folium.Marker(
#    location=[45.3311, -121.7113],
#    popup='Timberline Lodge',
#    icon=folium.Icon(color='green')
#).add_to(m)


In [ ]:
mapaguarderias = etiquetar(guarderias3.latitud,guarderias3.longitud,guarderias3.name)

In [84]:
def buscarLugaresCerca(address, maxDist=3000):
    point = geocode(address)
    return {
       "Lugares": {
         "$near": {
           "$geometry": point,
           "$maxDistance": maxDist,
         }
       }
    }

In [122]:
list(df_aeropuertos3.Lugares)

[{'type': 'Point', 'coordinates': [38.19605, -1.11457]},
 {'type': 'Point', 'coordinates': [40.39819, -3.71744]},
 {'type': 'Point', 'coordinates': [41.65525, -0.88338]}]

In [258]:
query_busqueda={ "Lugares": {
         "$near": {
           "$geometry": {'type': 'Point',
    'coordinates': [ 40.75677,-73.98964]},
           "$maxDistance": 10000,
         }
       }}


In [270]:
query_busquedaempresa={ "Lugares": {
         "$near": {
           "$geometry": {'type': 'Point',
    'coordinates': [-73.98964, 40.75677]},
           "$maxDistance": 1000,
         }
       }}


In [269]:
query = buscarLugaresCerca("Times Square New york")
query

<Response [200]>


{'Lugares': {'$near': {'$geometry': {'type': 'Point',
    'coordinates': [-73.98626, 40.75662]},
   '$maxDistance': 2000}}}

In [271]:
ver =db.NewYorkGamers.find(query_busquedaempresa)
empresasResultados =list(ver)
empresasResultados

[{'_id': ObjectId('5e9dd3c99c849366b3d04b88'),
  'index': 16341,
  'name': 'Real Time Content',
  'category_code': 'advertising',
  'Lugares': {'type': 'Point', 'coordinates': [-73.990286, 40.755959]},
  'clean_state': 'success',
  'founded_year': 2006.0,
  'latitude': 40.755959,
  'longitude': -73.990286},
 {'_id': ObjectId('5e9dd3c99c849366b3d04b9b'),
  'index': 1002,
  'name': 'ideeli',
  'category_code': 'ecommerce',
  'Lugares': {'type': 'Point', 'coordinates': [-73.990396, 40.755978]},
  'clean_state': 'success',
  'founded_year': 2007.0,
  'latitude': 40.755978,
  'longitude': -73.990396},
 {'_id': ObjectId('5e9dd3c99c849366b3d04b96'),
  'index': 14022,
  'name': 'Magnetic',
  'category_code': 'advertising',
  'Lugares': {'type': 'Point', 'coordinates': [-73.990152, 40.758343]},
  'clean_state': 'success',
  'founded_year': 2008.0,
  'latitude': 40.758343,
  'longitude': -73.990152},
 {'_id': ObjectId('5e9dd3c99c849366b3d04b8a'),
  'index': 13052,
  'name': 'MindSmack',
  'categ

In [248]:
verrest =db.NewYorkRestaurantes.find(query_busqueda)
restaurantesResult=list(verrest)
restaurantesResult

[{'_id': ObjectId('5e9e0a6c305ea4b69d874e74'),
  'name': 'MTHR Vegan',
  'adress': '329 W 49th St',
  'type': 'Point',
  'coordinates': [40.76201, -73.9874],
  'latitude': -73.9874,
  'longitude': 40.76201,
  'Lugares': {'type': 'Point', 'coordinates': [40.76201, -73.9874]}}]

In [250]:
verguar =db.NewYorkguarderias.find(query_busqueda)
guarderiasResult=list(verguar)


0


/home/sergio/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """


[]

In [ ]:
df_conGeoF = df_conGeo[["name", "Lugares"]]
df_guarderiasF = df_guarderias3[["name",  "Lugares"]]
df_starbucksF = df_starbucks3[["name",  "Lugares"]]
df_aeropuertosF = df_aeropuertos3[["name",  "Lugares"]]
display(df_conGeoF.head(3))
display(df_guarderiasF)
display(df_starbucksF)
display(df_aeropuertosF)

In [ ]:
df_guarderiasF.to_json("./Input/guarderiasfinal.json",orient="records")

In [ ]:
df_starbucksF.to_json("./Input/starbucksfinal.json",orient="records")
df_aeropuertosF.to_json("./Input/aeropuertosfinal.json",orient="records")

In [ ]:
df_conGeoF.to_json("./Input/aconGeofinal.json",orient="records")

In [ ]:
query

In [ ]:
if elem in df_aeropuertosF.Lugares:
    print(elem)